In [17]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.callbacks import Callback ,EarlyStopping,ModelCheckpoint
from sklearn.metrics import precision_score, recall_score,accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
df = pd.read_csv(r"C:\Users\admin\Desktop\Major project\dataset\cleanDataOne.csv")


In [19]:
df['text']

0             cocksucker before you piss around on my work
1        bye do not look come or think of comming back ...
2                   fuck your filthy mother in the ass dry
3        get fucked up get fuckeeed up  got a drink tha...
4        stupid peace of shit stop deleting my stuff as...
                               ...                        
18049    nothing of the sort  sumple and i are merely t...
18050    actually i broke a bone before  twice actually...
18051    ankit fadia study award updated the source for...
18052                           your sources are worthless
18053    this was the first time for me  it was very hi...
Name: text, Length: 18054, dtype: object

In [20]:
df

,text,y
0,cocksucker before you piss around on my work,1
1,bye do not look come or think of comming back ...,1
2,fuck your filthy mother in the ass dry,1
3,get fucked up get fuckeeed up got a drink tha...,1
4,stupid peace of shit stop deleting my stuff as...,1
...,...,...
18049,nothing of the sort sumple and i are merely t...,0
18050,actually i broke a bone before twice actually...,0
18051,ankit fadia study award updated the source for...,0
18052,your sources are worthless,0


In [21]:
X=df['text'].astype(str)
type(X)

pandas.core.series.Series

In [22]:

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=100)

In [23]:
import numpy as np

embeddings_index={}
with open(r'C:\Users\admin\Desktop\Major project\glove.6B.100d.txt','r',encoding='utf-8') as f:
  for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
embedding_dim=100
word_index=tokenizer.word_index


In [24]:
word_index

{'you': 1,
 'the': 2,
 'i': 3,
 'a': 4,
 'to': 5,
 'is': 6,
 'not': 7,
 'and': 8,
 'of': 9,
 'are': 10,
 'it': 11,
 'do': 12,
 'that': 13,
 'your': 14,
 'this': 15,
 'for': 16,
 'in': 17,
 'my': 18,
 'have': 19,
 'me': 20,
 'on': 21,
 'fuck': 22,
 'be': 23,
 'am': 24,
 'what': 25,
 'with': 26,
 'will': 27,
 'as': 28,
 'was': 29,
 'so': 30,
 'page': 31,
 'if': 32,
 'talk': 33,
 'just': 34,
 'like': 35,
 'please': 36,
 'an': 37,
 'fucking': 38,
 'from': 39,
 'all': 40,
 'article': 41,
 'wikipedia': 42,
 'did': 43,
 'go': 44,
 'can': 45,
 'about': 46,
 'no': 47,
 'or': 48,
 'get': 49,
 'but': 50,
 'why': 51,
 'up': 52,
 'stop': 53,
 'its': 54,
 'at': 55,
 'thanks': 56,
 'he': 57,
 'know': 58,
 'by': 59,
 'there': 60,
 'shit': 61,
 'how': 62,
 'off': 63,
 'who': 64,
 'think': 65,
 'should': 66,
 'now': 67,
 'edit': 68,
 'one': 69,
 'here': 70,
 'would': 71,
 'see': 72,
 'has': 73,
 'they': 74,
 'suck': 75,
 'out': 76,
 'blocked': 77,
 'some': 78,
 'want': 79,
 'we': 80,
 'because': 81,
 'h

In [25]:
embedding_matrix=np.zeros((len(word_index)+1,embedding_dim))
for word,i in word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector


In [26]:
len(word_index)

26610

In [27]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.49886   ,  0.76602   ,  0.89750999, ..., -0.41179001,
         0.40538999,  0.78504002],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ...,
       [ 0.30746001,  0.0071693 , -0.40382999, ...,  0.16892   ,
        -0.75593001, -0.13414   ],
       [ 0.050396  , -0.43832001, -0.26798999, ...,  0.019003  ,
        -0.6778    , -0.30882999],
       [ 0.12239   ,  0.40292999, -0.42792001, ...,  0.57059002,
        -0.23425999,  0.17585   ]])

In [28]:
precision = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')
early_stop=EarlyStopping(monitor='val_loss',patience=5)
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)


In [29]:
modelLstmBidirectionalGlove = Sequential()
modelLstmBidirectionalGlove.add(Embedding(input_dim=len(word_index)+1, output_dim=100,weights=[embedding_matrix],trainable=False))

modelLstmBidirectionalGlove.add(Bidirectional(LSTM(64)))
modelLstmBidirectionalGlove.add(Dense(1, activation='sigmoid'))
modelLstmBidirectionalGlove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',precision,recall])




In [30]:
x_train, x_val, y_train, y_val = train_test_split(X, df['y'], test_size=0.2, random_state=42)

In [31]:
history=modelLstmBidirectionalGlove.fit(x_train, y_train,validation_data=(x_val,y_val), epochs=30,batch_size=32,callbacks=[early_stop,checkpoint])


Epoch 1/30
451/452 [============================>.] - ETA: 0s - loss: 0.3562 - accuracy: 0.8515 - precision: 0.8610 - recall: 0.8383
Epoch 1: val_loss improved from inf to 0.28400, saving model to model_weights.h5
452/452 [==============================] - 32s 60ms/step - loss: 0.3560 - accuracy: 0.8516 - precision: 0.8612 - recall: 0.8384 - val_loss: 0.2840 - val_accuracy: 0.8829 - val_precision: 0.8958 - val_recall: 0.8666
Epoch 2/30
451/452 [============================>.] - ETA: 0s - loss: 0.2703 - accuracy: 0.8898 - precision: 0.8968 - recall: 0.8808
Epoch 2: val_loss improved from 0.28400 to 0.26254, saving model to model_weights.h5
452/452 [==============================] - 29s 63ms/step - loss: 0.2702 - accuracy: 0.8898 - precision: 0.8969 - recall: 0.8809 - val_loss: 0.2625 - val_accuracy: 0.8876 - val_precision: 0.9005 - val_recall: 0.8715
Epoch 3/30
452/452 [==============================] - ETA: 0s - loss: 0.2456 - accuracy: 0.8983 - precision: 0.9045 - recall: 0.8906
Epoch

In [32]:
lstm_bidirectional_Glove_pickle_data={'data':history.history,'model':modelLstmBidirectionalGlove}

In [34]:
lstm_bidirectional_Glove_pickle_data

{'data': {'loss': [0.35602903366088867,
   0.2701985538005829,
   0.24561920762062073,
   0.23042082786560059,
   0.21389876306056976,
   0.19824355840682983,
   0.18293778598308563,
   0.17027030885219574,
   0.154354065656662,
   0.13888688385486603,
   0.1224651113152504,
   0.11012469977140427],
  'accuracy': [0.8516235947608948,
   0.8898428082466125,
   0.8982897996902466,
   0.9067368507385254,
   0.9138683080673218,
   0.9216921925544739,
   0.9261926412582397,
   0.9345011711120605,
   0.9405940771102905,
   0.9483487010002136,
   0.9529876112937927,
   0.9595651626586914],
  'precision': [0.8611664175987244,
   0.8969261050224304,
   0.9045007228851318,
   0.9126158952713013,
   0.9184988141059875,
   0.9255170226097107,
   0.9304797649383545,
   0.9387498497962952,
   0.9441574811935425,
   0.9509680867195129,
   0.9565885066986084,
   0.9626376628875732],
  'recall': [0.8383880257606506,
   0.8809029459953308,
   0.890596866607666,
   0.8995984196662903,
   0.90832293033599

In [37]:
accuracy= sum(lstm_bidirectional_Glove_pickle_data['data']['accuracy'])/len(lstm_bidirectional_Glove_pickle_data['data']['accuracy'])
accuracy

0.920353576540947

In [33]:
import pickle

# Save the trained model object to a file
with open(r'C:\Users\admin\Desktop\Major project\modelLstmBidirectionalGlove.pkl', 'wb') as f:
    pickle.dump(lstm_bidirectional_Glove_pickle_data, f)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\bidirectional
......vars
...layers\bidirectional\backward_layer
......vars
...layers\bidirectional\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\forward_layer
......vars
...layers\bidirectional\forward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\layer
......vars
...layers\bidirectional\layer\cell
......vars
...layers\dense
......vars
.........0
.........1
...layers\embedding
......vars
.........0
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...metrics\precision
......vars
.........0
.........1
...metrics\recall
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Ker